In [1]:
import os

import pandas as pd
import deepchem as dc

import pandas as pd

from tqdm import tqdm
import json
import time
import csv
import os
import random

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from scipy.spatial.distance import cosine
from scipy.stats import pearsonr
import statsmodels.api as sm


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
/Users/lyu/anaconda3/envs/transKnow/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


chem_vocab = pd.read_csv("./CTD/CTD_chemicals.csv", comment='#', low_memory=False)
chem_vocab["ChemicalID"] = chem_vocab["ChemicalID"].str.extract(r'(C\d+)')
chem_vocab

In [2]:
chem_dis = pd.read_csv("./CTD/CTD_chemicals_diseases.csv", comment='#', low_memory=False)
chem_dis["DiseaseID"] = chem_dis["DiseaseID"].str.extract(r'(D\d+)')
chem_dis.head()

,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DirectEvidence,InferenceGeneSymbol,InferenceScore,OmimIDs,PubMedIDs
0,06-Paris-LA-66 protocol,C046983,NaN,Precursor Cell Lymphoblastic Leukemia-Lymphoma,D054198,therapeutic,NaN,NaN,NaN,4519131
1,10074-G5,C534883,NaN,Adenocarcinoma,D000230,NaN,MYC,4.07,NaN,26432044
2,10074-G5,C534883,NaN,Adenocarcinoma of Lung,D000077192,NaN,MYC,4.30,NaN,26656844|27602772
3,10074-G5,C534883,NaN,Alopecia,D000505,NaN,AR,4.49,NaN,15902657
4,10074-G5,C534883,NaN,Androgen-Insensitivity Syndrome,D013734,NaN,AR,6.86,300068|312300,1303262|8281139


In [3]:
chem_names = chem_dis["ChemicalName"].unique()
len(chem_names)

17337

In [4]:
from urllib.request import urlopen
from urllib.parse import quote


def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return False


identifiers = ['06-Paris-LA-66 protocol']

identifiers = []
smiles = []


In [5]:
identifiers_l = os.listdir("./Identifier")
print(identifiers_l)
id_s = []
for i in identifiers_l:
    id_s.extend(list(pd.read_csv(os.path.join("./Identifier/", i)).iloc[:, 0]))
id_s = list(set(id_s))
len(id_s)

['identifiers_pre.csv', 'identifiers.csv', 'Chem_id.csv', 'identifiers_8.csv', 'identifiers_9.csv', 'identifiers_4.csv', 'identifiers_5.csv', 'identifiers_7.csv', 'identifiers_6.csv', 'identifiers_2.csv', 'identifiers_3.csv', 'identifiers_1.csv', 'identifiers_12.csv', 'identifier.csv', 'identifiers_10.csv', 'identifiers_11.csv', 'identifier_c.csv']


EmptyDataError: No columns to parse from file

In [7]:
f = open("./Identifier/identifiers_13.csv", 'a', newline='')
csv_writer = csv.writer(f)

chem_names_o = [x for x in chem_names if x not in id_s]
random.shuffle(chem_names_o)
print(len(chem_names_o))

identifiers = []
smiles = []

for i, ids in enumerate(chem_names_o):

    ans = CIRconvert(ids)

    if ans:
        identifiers.append(ids)
        smiles.append(ans)
        csv_writer.writerow([ids, ans])
        f.flush()

    if i % 100 == 0 and i > 0:
        time.sleep(120)

    print(i, len(smiles), ids, ans)

f.close()

9561
0 0 reproterol False
1 0 BL 11282 False
2 0 cumene False
3 0 chiglitazar False
4 0 Tyrosine False
5 0 Pectins False
6 0 flavokawain B False
7 0 trandolapril False
8 0 Puromycin Aminonucleoside False
9 0 polihexanide False
10 0 HFPO4 compound False
11 0 iodosulpride False
12 0 pluronic block copolymer p85 False
13 0 D-161 compound False
14 0 salvigenin False
15 0 Barbiturates False
16 0 Cortodoxone False
17 0 naphthalenediimide False
18 0 Aminacrine False
19 0 N-(4-chloro-3-methoxyphenyl)-2-picolinamide False
20 0 KW 2581 False
21 0 tucatinib False
22 0 gepirone False
23 0 11-Mrp-14-Nal-18-Cys-22-Asp-beta-MSH(11-22)NH2 False
24 0 Cyclohexanes False
25 0 Desmodium gangeticum extract False
26 0 Kanamycin False
27 0 Danshen-Gegen False
28 0 trans-3,5,4'-trihydroxystilbene-4'-O-beta-D-glucopyranoside False
29 0 HDN-1 compound False
30 0 martentoxin, Buthus martensi False
31 0 CGC 11093 False
32 0 phenylhydrazonopyrazolone sulfonate 1 False
33 0 Glutarates False
34 0 Anesthetics False
3

In [7]:
f.close()

In [2]:
identifiers_l = os.listdir("./Identifier")
iu2smile={}
for i in identifiers_l:
    df_o=pd.read_csv(os.path.join("./Identifier/", i))
    for idx, row in df_o.iterrows():
        iu2smile[row.iloc[0]]=row.iloc[1]
len(iu2smile.keys())

iupac=[]
smiles=[]
for i in iu2smile.keys():
    iupac.append(i)
    smiles.append(iu2smile[i])
pd.DataFrame({
    'iupac':iupac,
    'smiles':smiles
}).to_csv('./Identifier/Chem_id.csv',index=False)

len(iupac)

7776

In [ ]:

f = open("./CTD/Identifiers.csv", 'w', encoding='utf8')
for i, ids in enumerate(chem_names):

    ans = CIRconvert(ids)
    if ans:
        identifiers.append(ids)
        smiles.append(ans)

    if i % 100 == 0 and i > 1:
        time.sleep(60)
    print(i, ids, ans)

0 06-Paris-LA-66 protocol False
1 10074-G5 False
2 10,10-bis(4-pyridinylmethyl)-9(10H)-anthracenone O=C1c2ccccc2C(Cc3ccncc3)(Cc4ccncc4)c5ccccc15
3 10,11-dihydro-10,11-dihydroxy-5H-dibenzazepine-5-carboxamide NC(=O)N1C=CC2=C(C(O)C(O)C=C2)c3ccccc13
4 10,11-dihydro-10-hydroxycarbamazepine NC(=O)N1c2ccccc2CC(O)c3ccccc13
5 10,11-dihydro-5H-dibenzo(a,d)cycloheptene C1Cc2ccccc2Cc3ccccc13
6 10,11-dihydroxy-N-n-propylnorapomorphine False
7 10,12-octadecadienoic acid CCCCCC=CC=CCCCCCCCCC(O)=O
8 10-(2-pyrazolylethoxy)camptothecin False
9 103D5R False
10 10-(4'-(N-diethylamino)butyl)-2-chlorophenoxazine False
11 10-(alpha-diethylaminopropionyl)phenothiazine CCN(CC)C(C)C(=O)N1c2ccccc2Sc3ccccc13
12 10-carboxymethyl-9-acridanone OC(=O)CN1c2ccccc2C(=O)c3ccccc13
13 10-deacetylpaclitaxel False
14 10-decarbamoylmitomycin C CO[C@@]12[C@@H]3N[C@@H]3CN1C4=C([C@H]2CO)C(=O)C(=C(C)C4=O)N
15 10-(fluoroethoxyphosphinyl)-N-(biotinamidopentyl)decanamide False
16 10-formyltetrahydropteroylglutamic acid NC1=Nc2ncc(C

In [10]:
for i, ids in enumerate(chem_names):

    if i < 700:
        continue

    ans = CIRconvert(ids)
    if ans:
        identifiers.append(ids)
        smiles.append(ans)

    if i % 100 == 0 and i > 1:
        time.sleep(90)
    print(i, ids, ans)

700 1-nitroso-5,6-dihydrouracil O=NN1CCC(=O)NC1=O
701 1-nitrosocyclohexyl acetate CC(=O)OC1(CCCCC1)N=O
702 1-nitrosopyrene O=Nc1ccc2ccc3cccc4ccc1c2c34
703 1-N-methyl-5-thiotetrazole CN1NN=NC1=S
704 1NMPP1 compound False
705 1-nonanol CCCCCCCCCO
706 1-O-(2-(2,3-dimethylphenyl)aminobenzoyl)glucopyranuronic acid Cc1cccc(Nc2ccccc2C(=O)OC3O[C@@H]([C@@H](O)[C@H](O)[C@H]3O)C(O)=O)c1C
707 1-O-(2-((2',6'-dichlorophenyl)amino)phenylacetyl)glucopyranuronic acid False
708 1-Octanol CCCCCCCCO
709 1-octen-3-ol CCCCCC(O)C=C
710 1-O-hexadecyl-2-N-methylcarbamylphosphatidylcholine CCCCCCCCCCCCCCCCOCC(CO[P]([O-])(=O)OCC[N+](C)(C)C)OC(=O)NC
711 1-O-hexyl-2,3,5-trimethylhydroquinone CCCCCCOc1cc(C)c(O)c(C)c1C
712 1-oleoyl-2-acetoyl-sn-glycerol CCCCCCCC\C=C/CCCCCCCC(=O)OC[C@H](CO)OC(C)=O
713 1-oleoyl-2-acetylglycerol CCCCCCCC\C=C/CCCCCCCC(=O)OCC(CO)OC(C)=O
714 1-oleoyl lysophosphatidylcholine CCCCCCCC\C=C/CCCCCCCC(=O)OCC(O)CO[P]([O-])(=O)OCC[N+](C)(C)C
715 1-oxo-1-(5-(2-pyridyl)-2-yl)-7-phenylheptane False


KeyboardInterrupt: 

In [7]:
len(identifiers)

343

In [12]:
pd.DataFrame({
    'Identifier': identifiers,
    'Smiles': smiles
}).to_csv('identifiers.csv', index=False)

In [8]:
chem_gene = pd.read_csv("./CTD/CTD_chem_gene_ixns.csv", comment='#', low_memory=False)
chem_gene.head()

,ChemicalName,ChemicalID,CasRN,GeneSymbol,GeneID,GeneForms,Organism,OrganismID,Interaction,InteractionActions,PubMedIDs
0,10074-G5,C534883,NaN,AR,367,protein,Homo sapiens,9606.0,10074-G5 affects the reaction [MYC protein res...,affects^reaction|increases^expression,32184358
1,10074-G5,C534883,NaN,AR,367,protein,Homo sapiens,9606.0,10074-G5 inhibits the reaction [EPHB2 protein ...,decreases^reaction|increases^expression,32184358
2,10074-G5,C534883,NaN,AR,367,protein,Homo sapiens,9606.0,10074-G5 results in decreased expression of AR...,decreases^expression,32184358
3,10074-G5,C534883,NaN,AR,367,protein,Homo sapiens,9606.0,10074-G5 results in decreased expression of AR...,decreases^expression,32184358
4,10074-G5,C534883,NaN,EPHB2,2048,protein,Homo sapiens,9606.0,10074-G5 inhibits the reaction [EPHB2 protein ...,decreases^reaction|increases^expression,32184358


In [19]:
pubMed_ids = []
for s in chem_dis['PubMedIDs']:
    t = str(s).split('|')
    pubMed_ids += t
pubMed_ids = list(set(pubMed_ids))

with open('pubMed_ids.json', 'w') as json_file:
    json.dump(pubMed_ids, json_file)

In [20]:
len(pubMed_ids)

90229

In [4]:
df_pid = pd.read_csv("./df_pid.csv")
df_pid

,PMID,Abstract
0,1276554,Hyperkinetic movement disorders may occur as s...
1,1276935,Arginine and large doses of glutamate (greater...
2,1276991,A study was conducted to assess whether the pr...
3,1277139,The effect of vitamin A deficiency and hypervi...
4,1277158,N-[4-(5-Nitro-2-furyl)-2-thiazolyl]formamide (...
...,...,...
20905,32754917,Staphylococcal enterotoxin-B (SEB) is one of t...
20906,32754917,Efficacy of ? -tetrahydrocannabinol (THC) was ...
20907,32754917,While SEB triggered ARDS and 100% mortality in...
20908,32754917,The study demonstrates for the first time that...
